In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

# Get the tipping points for scenario 4

In [ ]:
scenario = "2025-07-08_4"
dir = "case_study_eurostar_pc"
filename = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", dir, scenario, "eurostar.csv")
df = pd.read_csv(filename)
df

In [ ]:
cities = {
    "Gv": "The Hague",
    "Dt": "Delft",
    "Dtcp": "Delft",
    "Laa": "The Hague",
    "Rtd": "Rotterdam",
    "Shl": "Schiphol",
    "Hfd": "Hoofddorp",
    "Ledn": "Leiden",
    "Sdm": "Schiedam",
    "Rmoa": "Rotterdam",
    "Gvm": "The Hague"
}

In [ ]:
allowed_delays = [180]

def td_str(td):
    return ':'.join(re.split(r'[:.]+', str(td)) [1:3])

def extract_tipping_point(df, show_column="Delay Location"):
    def apply_func(df):
        result=df.groupby(show_column).agg({
            "beta": "max",
            "Delay Amount": "max",
        })
        result["Tipping Point"] = result["beta"].apply(lambda x: td_str(timedelta(seconds=x)))
        result["Delay Amount"] = result["Delay Amount"].apply(lambda x: td_str(timedelta(seconds=x)))
        return result.sort_values("Tipping Point", ascending=True).drop(columns=["beta"])
    df["Delay Location"] = df["Delay Location"].apply(lambda x: x.split("|")[0])
    df["Location"] = df["Delay Location"].apply(lambda x: cities[x.split("_")[0]] if x != "" and x != "-" else x)
    df = df.groupby(by=['Train', 'Train ID']).apply(apply_func, include_groups=False)
    return df

tp_df = df.rename(columns={
    "delay_amount": "Delay Amount",
    "delay_location": "Delay Location",
    "trainNumber": "Train",
    "id": "Train ID",
    "scenario": "Scenario",
    "label": "Label",
})
tp_df["Scenario"] = tp_df["Scenario"].apply(lambda x: x.split(".")[0])
tp_df["Delay Location"] = tp_df["Delay Location"].apply(lambda x: x.split("-")[1])
# Only do one scenario
tp_df = tp_df[tp_df["Scenario"] == scenario]
tp_df = extract_tipping_point(tp_df, "Location")
tp_df

In [ ]:
lines = tp_df.to_latex().split("\n")
latex_filename = filename.replace("eurostar.csv", "table_tipping_points.tex")
new_lines = []
i = 0
while i < len(lines):
    if "\begin{tabular}" in lines[i]:
        new_lines.append(r'\begin{tabular}{llll}')
    elif "cline" in lines[i]:
        pass
    elif lines[i] == r'\toprule':
        new_lines.append(r'\toprule')
        new_lines.append(r'Train & Location & Delay & Tipping point \\')
        i += 2
    elif r'&' in lines[i]:
        # Remove first column which has train number
        new_lines.append(r' & '.join(lines[i].split(r' & ')[1:]).replace(".000000", ""))
    else:
        new_lines.append(lines[i])
    i += 1
for line in new_lines:
    print(line)
with open(latex_filename, "w") as f:
    f.write("\n".join(new_lines))

In [ ]:
# dir = "case_study_eurostar_pc"
# folder = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", dir)
# filename_all = "eurostar_all.csv"
# all_lines = []
# for j, scenario in enumerate(os.listdir(folder)):
#     filename = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", dir, scenario, "eurostar.csv")
#     if os.path.isfile(filename):
#         tmp_df = pd.read_csv(filename)
#         tmp_df = tmp_df.rename(columns={"delay_amount": "Delay Amount", "delay_location": "Delay Location","trainNumber": "Train","id": "Train ID","scenario": "Scenario","label": "Label",})
#         tmp_df["Scenario"] = tmp_df["Scenario"].apply(lambda x: x.split(".")[0])
#         tmp_df["Delay Location"] = tmp_df["Delay Location"].apply(lambda x: x.split("-")[1])
#         tmp_df = tmp_df[tmp_df["Scenario"] == scenario]
#         tmp_df = extract_tipping_point(tmp_df, "Location")
#         num_tps = tmp_df.groupby(["Label"])["Tipping Point"].count()
#         num_tps = num_tps.unstack("Label").rename("").astype(int)
#         # lines = tmp_df.to_latex().split("\n")
#         # i = 0
#         # while i < len(lines):
#         #     if "\begin{tabular}" in lines[i]:
#         #         if j == 0:
#         #             all_lines.append(r'\begin{tabular}{llll}')
#         #     elif "cline" in lines[i]:
#         #         pass
#         #     elif lines[i] == r'\toprule':
#         #         if j == 0:
#         #             all_lines.append(r'\toprule')
#         #             all_lines.append(r'Train & Location & Delay & Tipping point \\')
#         #         i += 2
#         #     elif r'&' in lines[i]:
#         #         # Remove first column which has train number and print train id as int
#         #         all_lines.append(r' & '.join(lines[i].split(r' & ')[1:]).replace(".000000", ""))
#         #     elif ("bottom" in lines[i] or "end" in lines[i]) and j == 3:
#         #         all_lines.append(lines[i])
#         #     elif "midrule" in lines[i] and j == 0:
#         #         all_lines.append(lines[i])
#         #     else:
#         #         print("check", lines[i])
#         #     i += 1
# for line in all_lines:
#     print(line)

# Tipping points VIRM

In [ ]:
scenario = "2025-07-08_3"

virm_filename = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", dir, scenario, "eurostar.csv")
# Input was 3500
virm_df = pd.read_csv(virm_filename)
vtp_df = virm_df.rename(columns={
    "delay_amount": "Delay Amount",
    "delay_location": "Delay Location",
    "trainNumber": "Train",
    "id": "Train ID",
    "scenario": "Scenario",
    "label": "Label",
})
vtp_df["Scenario"] = vtp_df["Scenario"].apply(lambda x: x.split(".")[0])
vtp_df["Delay Location"] = vtp_df["Delay Location"].apply(lambda x: x.split("-")[1])
# Only do one scenario
vtp_df = vtp_df[vtp_df["Scenario"] == scenario]
vtp_df = extract_tipping_point(vtp_df, "Delay Location")
vtp_df

# Comparison

In [ ]:
folder = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", dir)
filename_time = "eurostar-time_all.csv"
all_time_df = []
for scenario in os.listdir(folder):
    if os.path.isdir(os.path.join(folder, scenario)):
        all_time_df.append(pd.read_csv(os.path.join(folder, scenario, "eurostar-time.csv")))
frame_time = pd.concat(all_time_df, axis=0, ignore_index=True)
frame_time.to_csv(os.path.join(folder, filename_time))

In [ ]:
number_trains_250708 = {
    1: 64,
    2: 82,
    3: 77,
    4: 82
}

In [ ]:
all_filename = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", dir, "test_all.csv")
comp_df = pd.read_csv(all_filename, index_col=None)
comp_df["Scenario"] = comp_df["scenario"].apply(lambda x: int(x.split("_")[1].split(".")[0])).astype(int)
comp_df["Trains"] = comp_df["Scenario"].apply(lambda x: number_trains_250708[x]).astype(int)
comp_df["Location"] = comp_df["delay_location"].apply(lambda x: cities[x.split("|")[0].split("-")[1].split("_")[0]] if x != "-" else x)
res = comp_df.groupby(["Scenario", "label"])["path"].count()
res = res.unstack("label").astype(int)
lines = res[["FlexSIPP", "@MAEDeR", "rSIPP"]].to_latex().split('\n')
i = 0
new_lines = []
while i < len(lines):
    if "toprule" in lines[i]:
        new_lines.append(lines[i])
        new_lines.append(lines[i+1].replace("label", "Scenario"))
        i += 2
    else:
        new_lines.append(lines[i])
    i += 1
with open(all_filename.replace("test_all.csv", "table_comparison.tex"), "w") as f:
    f.write("\n".join(new_lines))


In [ ]:
# allowed_delays = [180]

# def extract_tipping_point_all_scenarios(df):
#     def apply_func(df):
#         result=df.groupby("Delay Location").agg({
#             "alpha": "min",
#             "beta": "max",
#             "Delay Amount": "max"
#         })
#         result = result.loc[result['alpha'] < 900]
#         result["Tipping Point"] = result.apply(lambda x: td_str(timedelta(seconds=x['alpha'])) if x['alpha'] != x['beta'] else np.nan, axis=1)
#         return result.sort_values("Tipping Point", ascending=True).drop(columns=["alpha", "beta"])
#     df["Delay Location"] = df["Delay Location"].str.split("|").apply(lambda x: x[0])
#     df = df.groupby(by='Train ID').apply(apply_func, include_groups=False)
#     return df

# ctp_df = comp_df.rename(columns={
#     "delay_amount": "Delay Amount",
#     "delay_location": "Delay Location",
#     "id": "Train ID",
#     "label": "Label",
# })
# ctp_df = ctp_df.groupby(["Scenario", "Label"]).apply(extract_tipping_point_all_scenarios, include_groups=False).reset_index()
# num_tps = ctp_df.groupby(["Scenario", "Label"])["Tipping Point"].count()
# num_tps = num_tps.unstack("Label").rename("").astype(int)
# num_tps

In [ ]:
# num_lines = num_tps.to_latex().split("\n")
# num_filename = all_filename.replace("eurostar_all.csv", "table_comparison.tex")
# new_lines_num = []
# i = 0
# while i < len(num_lines):
#     if num_lines[i] == r'\toprule':
#         new_lines_num.append(r'\toprule')
#         new_lines_num.append(num_lines[i+1].replace(r' & Label', r'Scenario & Trains'))
#         i += 2
#     elif "cline" in num_lines[i]:
#         pass
#     else:
#         new_lines_num.append(num_lines[i])
#     i += 1
# for line in new_lines_num:
#     print(line)
# with open(num_filename, "w") as f:
#     f.write("\n".join(new_lines_num))

In [ ]:
# folder = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", dir)
# filename_all = "eurostar_all_together.csv"
# all_lines = []
# all_time_df = []
# for scenario in os.listdir(folder):
#     if os.path.isdir(os.path.join(folder, scenario)):
#         tmp_df = pd.read_csv(os.path.join(folder, scenario, "eurostar.csv"))
#         tmp_df = tmp_df.reset_index(drop=True)
#         tmp_df["Scenario"] = tmp_df["scenario"].apply(lambda x: int(x.split("_")[1].split(".")[0])).astype(int)
#         tmp_df["Trains"] = tmp_df["Scenario"].apply(lambda x: number_trains_250708[x]).astype(int)
#         tmp_df["Location"] = tmp_df["delay_location"].apply(lambda x: cities[x.split("|")[0].split("-")[1].split("_")[0]] if x != "-" else x)
#         # tmp_df["Location2"] = tmp_df["delay_location"].apply(lambda x: cities[x.split("_")[0]] if x != "" and x != "-" else x)
#         print(tmp_df["Location"].unique())
#         tmp_df = tmp_df.rename(columns={
#             "delay_amount": "Delay Amount",
#             "delay_location": "Delay Location",
#             "id": "Train ID",
#             "trainNumber": "Train",
#             "label": "Label",
#         })
#         tmp_df["Delay Location"] = tmp_df["Delay Location"].apply(lambda x: x.split("-")[1])
#         tmp_df = tmp_df.groupby(["Label"]).apply(extract_tipping_point, include_groups=False).reset_index()
# #         num_tps = ctp_df.groupby(["Scenario", "Label"])["Tipping Point"].count()
# #         num_tps = num_tps.unstack("Label").rename("").astype(int)
# # num_tps

# Runtime results

In [ ]:
time_components = {
    "unsafe interval generation": "Conflict gen.",
    "safe interval generation": "Interval gen.",
    "bt and crt generation": "Flexibility gen.",
    "converting routes to blocks": "Block conversion",
    "track graph creation": "Track graph creation",
    "routing graph creation": "Routing graph creation",
}
time_order = [
    r'Creation Routes~$\network$',
    "Conflict gen.",
    "Interval gen.",
    "Flexibility gen.",
    "Search FlexSIPP",
    "Search @MAEDeR",
    "Search rSIPP"
]
algorithms_ordered = ["FlexSIPP", "@MAEDeR", "rSIPP"]
column_order = [(s,a) for s in number_trains_250708.keys() for a in algorithms_ordered]

In [ ]:
time_filename = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", dir, "eurostar-time_all.csv")
form = "tex"
time_result = time_filename.replace("eurostar-time_all.csv", f"runtime_components.{form}")
time_df = pd.read_csv(time_filename)
time_df[r'Creation Routes~$\network$'] = time_df.apply(lambda x: x["routing graph creation"] + x["track graph creation"] + x["converting routes to blocks"], axis=1)
time_df["Scenario (num)"] = time_df["scenario"].apply(lambda x: int(x.split("_")[1].split(".")[0]))
time_df["Scenario"] = time_df["scenario"].apply(lambda x: f"Scen {x.split('_')[1].split('.')[0]}")
time_df["Search FlexSIPP"] = time_df.apply(lambda x: x["FlexSIPP search time"] if x["label"] == "FlexSIPP" else np.nan, axis=1)
time_df["Search @MAEDeR"] = time_df.apply(lambda x: x["FlexSIPP search time"] if x["label"] == "@MAEDeR" else np.nan, axis=1)
time_df["Search rSIPP"] = time_df.apply(lambda x: x["FlexSIPP search time"] if x["label"] == "rSIPP" else np.nan, axis=1)
for (old, new) in time_components.items():
    time_df[new] = time_df[old]
pv = time_df.pivot_table(
    columns = ["Scenario"],
    values = time_order,
    aggfunc="mean",
    fill_value=0
)
pv = pv.loc[time_order]
pv

In [ ]:
time_lines = pv.to_latex(float_format="%.1f").split("\n")
print(time_lines)
i = 0
new_time_lines = []
while i < len(time_lines):
    if r'\toprule' in time_lines[i]:
        new_time_lines.append(time_lines[i])
        new_time_lines.append(time_lines[i+1].replace("Scenario &", "Time Components &"))
        i += 1
    else:
        new_time_lines.append(time_lines[i])
    i += 1
for line in new_time_lines:
    print(line)
with open(time_result, "w") as f:
    f.write('\n'.join(new_time_lines))

# Old graphs for time components

In [ ]:
pv = time_df.pivot_table(
    columns = ["Scenario (num)", "label"],
    values = time_order,
    aggfunc="mean",
    fill_value=0
)
ax = pv.T.plot(
    kind='bar',
    stacked=True,
    figsize=(10,6)

)
min_font=12
ax.set_xticklabels([x[1] for x in column_order], rotation=45, fontsize=min_font)
column_names = [f"Scenario {x}" * len(time_df["label"].unique()) for x in time_df["Scenario (num)"].unique()]
scen_name_pos = -0.2
for i, col_name in enumerate(number_trains_250708.keys()):
    ax.text(
        i * len(time_df["label"].unique()) + 1,
        scen_name_pos,
        f"Scenario {col_name}",
        ha='center',
        va='top',
        transform=ax.get_xaxis_transform(),
        fontsize=min_font
    )
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], bbox_to_anchor=(0.5,scen_name_pos*2), loc="center", ncols=len(pv.index)/2, title="Time components", title_fontsize=min_font, fontsize=min_font)
plt.ylabel("Time in seconds", fontsize=min_font)
plt.yticks(fontsize=min_font)
plt.xlabel("")
plt.savefig(time_result, format=form, dpi=600)
plt.show()
pv

In [ ]:
group_space = 1.2
x = []
group = 0
for i, (col, typ) in enumerate(pv.columns):
    pos_in_group = i % len(algorithms_ordered)
    # base position + offset + spacing
    x.append(group * (len(algorithms_ordered) + group_space) + pos_in_group)
    # move to next group after every 2 bars
    if pos_in_group == len(algorithms_ordered) - 1:
        group += 1
x = np.array(x)

fig, ax = plt.subplots(figsize=(10,6))

bottom = np.zeros(len(x))
for row in time_order:
    ax.bar(x, pv.loc[row], bottom=bottom, label=row)
    bottom += pv.loc[row].values
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], bbox_to_anchor=(0.5,-0.3), loc="center", ncols=len(pv.index)/2, title="Time components")

ax.set_xticklabels([x[1] for x in column_order], rotation=45)
column_names = [f"Scenario {x}" * len(time_df["label"].unique()) for x in time_df["Scenario"].unique()]
for i, col_name in enumerate(number_trains_250708.keys()):
    ax.text(
        i * len(time_df["label"].unique()) + 1,
        -0.15,
        f"Scenario {col_name}",
        ha='center',
        va='top',
        transform=ax.get_xaxis_transform(),
        fontsize=11
    )


plt.ylabel("Time in seconds")
plt.xlabel("")
plt.yscale("log")
# plt.savefig(time_result, format=form, dpi=600)
plt.show()